## Bedrock モデルと Langchain エージェントの統合

特定のアプリケーションでは、ユーザー入力に応じて、言語モデルや各種ユーティリティへの呼び出しシーケンスを調整する必要があります。LangChain エージェントインターフェイスを使用すると、アプリケーションに柔軟性を持たせることができます。エージェントはさまざまなリソースを利用でき、ユーザーの入力に基づいて使用するリソースを選択します。エージェントは複数のツールを使用でき、あるツールの出力を別のツールの入力として使用できます。

エージェントには2つの主なカテゴリがあります:

- アクションエージェント:各ステップで、アクションエージェントは以前のすべてのアクションの出力を使用して後続のアクションを決定します。
- エージェントの計画と実行:これらのエージェントは、最初にアクションの完全な順序を決定し、プランを更新せずにアクションを実行します。

このノートブックでは、`Zero-shot ReAct` (これはアクションベースのエージェントで、ツールの説明のみに基づいて適切なツールを選択するために [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) フレームワークを使用します) とともに `plan-and-execute` エージェントの使用方法を確認します。そこでは、各ツールの説明を提供する必要があります。

## アーキテクチャ
![](./images/arch-agents.png)

In [ ]:
import json
import os
import sys
import boto3

# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

os.environ['SERPAPI_API_KEY'] = "<YOUR_SERP_API_KEY>" # https://serpapi.com/ で取得した API Key を設定してください


In [ ]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

## ReAct の使用: 言語モデルにおける推論と行動のシナジー

大規模な言語モデルは、推論に対する説明とタスクに特化した応答の両方を生成することができます。

推論の説明を生成することで、モデルはアクションプランを設定、監視、修正でき、予期しないシナリオにも対応できます。アクションステップにより、モデルはナレッジベースや環境などの外部ソースと通信し、情報を取得することができます。

ReAct フレームワークにより、大規模な言語モデルは外部ツールとインタラクションして追加情報を得ることができ、その結果、より正確で事実に基づいた応答が可能になります。

In [ ]:
from langchain.agents import load_tools
from langchain.agents import  AgentExecutor, create_react_agent, Tool
from langchain.agents import AgentType
from langchain_community.llms.bedrock import Bedrock
from langchain.chains.llm_math.base import LLMMathChain
from langchain_community.utilities.serpapi import SerpAPIWrapper

エージェント用と計算チェーン用に、それぞれ異なるモデルパラメータを持つ 2 つの固有の LLM オブジェクトをロードします。
計算チェーン用の固有の LLM と停止シーケンスにより、計算チェーンの実行中に Claude が冗長になるのを防ぐことができます。

さらに、デフォルトの LangChain テンプレートはデフォルトでは Claude に適合していないので、デフォルトのテンプレートをClaude に合わせて調整し、新しく構築したツールをツールリストに追加します。

In [ ]:
react_agent_llm = Bedrock(model_id="anthropic.claude-instant-v1", model_kwargs=model_parameter)
math_chain_llm = Bedrock(model_id="anthropic.claude-instant-v1",
                         model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

tools = load_tools(["serpapi"], llm=react_agent_llm)

llm_math_chain = LLMMathChain.from_llm(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)

In [ ]:
from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""
)

In [ ]:
agent = create_react_agent(
                            react_agent_llm, 
                            tools, 
                            prompt=prompt_template
                               )

In [ ]:
question = "What is Amazon SageMaker? What is the launch year multiplied by 2"

In [ ]:
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
executor.invoke({"input": question})

## ツールコードの生成
生成 AI を使用してツールのコードを作成できますか? はい、できます!   
ただし、コードにはある程度のバリエーションがあり、人間によるレビューなしに実行するのは安全ではない場合があるので注意が必要です。

次の例では、Claude によって生成されたコードを使用してツールを生成しています。EC2 インスタンスの検索ツールを boto3 python コードで実装しています。

In [ ]:
import xml.etree.ElementTree as ET
from IPython.display import display, Markdown, Latex

boto3_bedrock = boto3.client(service_name='bedrock-runtime',
                             region_name=os.environ["AWS_DEFAULT_REGION"])

prompt_data = """
Human: You are an AI python code generator. You write really great code.

Write a python function named list_tagged_instances with one parameter named tagname. 

The function queries the boto3 library to return a list all of the EC2 instances that have a tag key equal to the tagname parameter. 

return the code inside <code></code>
Assistant:"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

tree = ET.ElementTree(ET.fromstring(response_body.get("completion")))
python_code = tree.getroot().text

display(Markdown(f'```{python_code}```'))

exec(python_code)

tools = [
    Tool.from_function(
        func=list_tagged_instances,
        name="List",
        description="List all of the EC2 instances that have a tag equal to the tagname parameter."
    ),
]

## コードを実行する

上記のコードが適切だと思われる場合、エージェントのツールとして使用します。

**注:** コードを機能させるには、このアカウントに 'delete' という名前のタグとそのタグに任意の値を持つ EC2 インスタンスが必要です。大文字と小文字が区別されるため、これを実行する前に、EC2インスタンスを作成し、タグを設定してください。

In [ ]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

agent = create_react_agent(react_agent_llm, tools, prompt=prompt_template)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

question = """Please list my EC2 instances with the tag delete."""

result = executor.invoke({"input": question})

print(f"{result}")

## 別のシナリオ - 存在しないインスタンスを検索する
存在しないタグを指定し、 EC2 インスタンスが見つからないことを確認します。

In [ ]:
question = """Please list my EC2 instances with the tag doesnotexist."""

result = executor.invoke({"input": question})

print(f"{result}")

## Database Tool
エージェントの一般的な用途は、データベース内のレコードを検索することです。コンテキストに完全なデータベースを含めるのは現実的ではないので、会話型のインタラクションを維持しながら、データベースに対するアクションを実行して幻覚 (ハルシネーション) を排除するツールを提供できます。

### SQL データベースエージェント
LangaChain には、DB に質問して答えを得る方法をデモするための SQL データベースエージェントがあります。詳細は、このドキュメントを参照してください: https://python.langchain.com/docs/integrations/toolkits/sql_database

エージェントは DB のスキーマをコンテキストにロードし、自然言語の質問に基づいて SQL ステートメントを生成します。その後、SQL ステートメントがデータベースに対して実行され、結果が返されます。

### Data エージェント
SQL データベースエージェントは、データ探索やクエリ生成に役立ちます。また、次のステップのコンテキストをプロンプトで提供するために、データベースからデータをプルするツールを作成したい場合もあります。
次の例では、customer テーブル内の顧客の DB クエリをシミュレートします。このコードを DynamoDB またはリレーショナルデータベースの検索に置き換えてください。

In [ ]:
customer_table=[
  {
    "id": 1, 
    "first_name": "John", 
    "last_name": "Doe",
    "age": 35,
    "postal_code": "90210"
  },
  {  
    "id": 2,
    "first_name": "Jane",
    "last_name": "Smith", 
    "age": 27,
    "postal_code": "12345"
  },
  {
    "id": 3, 
    "first_name": "Bob",
    "last_name": "Jones",
    "age": 42,
    "postal_code": "55555"
  },
  {
    "id": 4,
    "first_name": "Sara", 
    "last_name": "Miller",
    "age": 29, 
    "postal_code": "13579"
  },
  {
    "id": 5,
    "first_name": "Mark",
    "last_name": "Davis",
    "age": 31,
    "postal_code": "02468"
  },
  {
    "id": 6,
    "first_name": "Laura",
    "last_name": "Wilson",
    "age": 24,
    "postal_code": "98765" 
  },
  {
    "id": 7,
    "first_name": "Steve",
    "last_name": "Moore",
    "age": 36,
    "postal_code": "11223"
  },
  {
    "id": 8,
    "first_name": "Michelle",
    "last_name": "Chen",
    "age": 22,
    "orders": [
        {
            "order_id": 1,
            "description": "An order of 1 dozen pencils"
        },
        {
            "order_id": 2,
            "description": "An order of 2 markers"
        }
    ],
    "postal_code": "33215"
  },
  {
    "id": 9,
    "first_name": "David",
    "last_name": "Lee",
    "age": 29,
    "postal_code": "99567"
  },
  {
    "id": 10,
    "first_name": "Jessica",
    "last_name": "Brown",
    "age": 18, 
    "postal_code": "43210"
  }
]

def customer_lookup(id):
    print(f"search by customer {id}")
    for customer in customer_table:
        if customer["id"] == int(id):
            print(f"found customer {id} {customer}")
            return customer
        
    return None

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))

In [ ]:
react_agent = create_react_agent(llm, tools, prompt_template)
executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True)

question = """write one sentence summary about the information you know about the customer with an id of 2."""

result = executor.invoke({"input": question})

print(f"{result}")